In [1]:
# set pyspark env variables
import os
os.environ['SPARK_HOME'] = "/Users/monkey/.sdkman/candidates/spark/current"
os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python3'

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

# Create a SparkSession
spark = SparkSession.builder \
    .appName("data-frames-intro") \
    .getOrCreate()

24/09/08 23:43:06 WARN Utils: Your hostname, Alexs-Mac-mini.local resolves to a loopback address: 127.0.0.1; using 192.168.178.87 instead (on interface en0)
24/09/08 23:43:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 23:43:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Using RDDs

In [3]:
rdd = spark.sparkContext.textFile("./data/data.txt")
result_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

In [4]:
result_rdd.take(10)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('distributed', 5),
 ('in', 5),
 ('Spark', 4),
 ('as', 3),
 ('is', 3),
 ('API', 3),
 ('on', 3)]

## Using DataFrames

In [5]:
df = spark.read.text("./data/data.txt")

result_df = df.selectExpr("explode(split(value, ' ')) as word") \
    .groupBy("word").count().orderBy(desc("count"))

In [6]:
result_df.take(10)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5),
 Row(word='Spark', count=4),
 Row(word='API', count=3),
 Row(word='RDD', count=3),
 Row(word='is', count=3),
 Row(word='on', count=3)]

In [7]:
spark.stop()